In [91]:
import re, os, requests
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "config.settings")

import django
django.setup()

from datetime import datetime
from stock.models import PresidentFact

from bs4 import BeautifulSoup


class PresidentNewsroomCrawling:
    def __init__(self):
        print('__init__')
        
        self.link_url = []
        
        self.president_url = requests.get(url='https://www.president.go.kr')
        self.president_newsroom_url = requests.get(url='https://www.president.go.kr/newsroom')
        self.president_newsroom_fact_url = requests.get(url='https://www.president.go.kr/newsroom/fact/')
        self.president_newsroom_briefing_url = requests.get(url='https://www.president.go.kr/newsroom/briefing/')
        self.headers = {"Content-Type": "application/json"}
        

    def crawler(self, target):
        if target == 'fact':
            html = self.president_newsroom_fact_url
            soup = BeautifulSoup(html.text, 'html.parser')
            bs = soup.select(
                "body > div.container > div > section.fact > div > div > a"
            )
            for b in bs:
                self.link_url.append(self.president_url.url.rstrip('/') + b.get('href'))
            
            for url in self.link_url:
                html = requests.get(url=url)
                soup = BeautifulSoup(html.text, 'html.parser')
                bs = soup.select(
                    "body > div.container > div > section.view_area"
                )

                for b in bs:
                    text = b.get_text().replace("\n", "").strip("복사확인").replace("\xa0", "")
                    
                    match = re.search(r'\d{4}.\d{2}.\d{2}', text)
                    date = datetime.strptime(match.group(), '%Y.%m.%d')  # match.group() 2023.01.27 to datetime
                    remove_dates = re.sub(r'\d{4}.\d{2}.\d{2}', '', text)  # text 에서 날짜 제거

                    get_info = remove_dates.split('→')

                    get_title = get_info[0]
                    get_description = get_info[1].split("목록")[0]
                    get_link = get_info[1].split("목록")[0].split("링크")[0]

                    PresidentFact.objects.update_or_create(
                        title=get_title,
                        description=get_description,
                        link=get_link,
                        date=date
                    )
            
        elif target == 'briefing':
            html = self.president_newsroom_briefing_url
        
p = PresidentNewsroomCrawling()
p.crawler('fact')

__init__
['https://www.president.go.kr/newsroom/fact/ndCQzWjq', 'https://www.president.go.kr/newsroom/fact/SsHyFqH1', 'https://www.president.go.kr/newsroom/fact/ZUgPX660', 'https://www.president.go.kr/newsroom/fact/hYdUZkDp', 'https://www.president.go.kr/newsroom/fact/lS0AWDSq']
['교육과정서 의도적으로 5.18 민주화운동 삭제? → "文정부 때 시안 마련"2023.01.04\'윤석열 정부가 교육과정에서 의도적으로 5.18 민주화운동을 삭제하였다\'는 주장은 전혀 사실이 아닙니다.\xa02022년 개정 교육과정에서 학습요소 항목이 생략되면서 5.18 민주화운동뿐 아니라 모든 개별적이고 구체적인 역사적 사건의 서술을 최소화하였습니다.\xa0문재인 정부 때인 2021년 12월에 구성되어 역사과 교육과정을 개발한 정책연구진도 이러한 취지에서 구체적인 역사적 사건 서술을 축소했습니다. 그래서 연구진이 교육부에 제출한 개정 교육과정의 최초 시안부터 5.18 민주화운동은 포함되어 있지 않았습니다.\xa0구체적 역사 사건명은 생략됐으나 현행 교육과정과 마찬가지로 2022년 개정 교육과정에서도 4.19 혁명에서 6월 민주항쟁에 이르는 민주화운동의 과정을 학습할 수 있도록 관련 성취 규정을 개발하고 있습니다.\xa0• 1월 3일 오마이뉴스 <[단독] 윤석열 정부, 개정 교육과정에서 \'5.18민주화운동\' 삭제>에 대한 대통령실의 설명입니다. 목록https://www.president.go.kr/newsroom/fact/ndCQzWjq링크 ', '대통령 연하장, 해외 이미지 도용? → "적법 계약 통해 K-문화 담아"2022.12.30일부 언론의 연하장 디자인 도용 의혹 제기는 전혀 사실이 아닙니다.\xa0새해 연하장은 역대 대통령의 연하장을 다수 제작한 경험